Your goal is to:
- Detect the face in the first frame of the movie
- Using pre-trained Viola-Jones detector
- Track the face throughout the movie using:
   - CAMShift
   - Particle Filter
   - Face detector + Kalman Filter (always run the kf.predict(), and run kf.correct() when you get a new face detection)
- Bonus (20pt): Face Detector + Optical Flow tracker (use the OF tracker whenever the face detector fails).


In [1]:
import os
import sys
import cv2
import matplotlib.pyplot as plt
import numpy as np

face_cascade = cv2.CascadeClassifier('/usr/local/share/OpenCV/haarcascades/haarcascade_frontalface_default.xml')

Debug_laptop = 1
filename = "02-1.avi"
if Debug_laptop:
    import skvideo.io
    v = skvideo.io.vread(filename)
else :
    v = cv2.VideoCapture(filename)

output_name = "output_of.txt"
output = open(output_name, "w")

/Users/trungnguyen/anaconda/lib/python2.7/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)


In [2]:
def detect_one_face(im):
    gray=cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.2, 3)
    if len(faces) == 0:
        return (0,0,0,0)
    return faces[0]

In [3]:
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.05,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))

# read first frame
old_frame = v[0]
frameCounter = 0

# detect face in first frame
c,r,w,h = detect_one_face(old_frame)
faces = (c,r,w,h)
pt = (frameCounter, c+w/2, r+h/2)
output.write("%d,%d,%d\n" % pt)  
frameCounter = frameCounter + 1

# Take first frame and find corners in it
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

roi = np.zeros(old_gray.shape)
roi[r:r+h,c:c+w] = 1

p0 = cv2.goodFeaturesToTrack(old_gray, mask = np.uint8(roi), **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while (1):
    if frameCounter == len(v): break
    
    frame = v[frameCounter]  # read frame
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    # Select good points    
    good_old = p0[st==1]
    good_new = p1[st==1]
    average_point = np.uint8(np.average(good_new, axis=0))
    pt = (frameCounter, average_point[0], average_point[1])
    output.write("%d,%d,%d\n" % pt)  
        
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
        
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    frameCounter = frameCounter + 1
    
cv2.destroyAllWindows()

output.close()